In [2]:
from pyknow import *
from random import choice

# PyKnow

[`pyknow`](https://github.com/buguroo/pyknow/) is a Python library for building expert systems

# Road crossing example

This example shows how simple agent given basic rules can behave in an environment which is a traffic light

Rules:
* If current color is `red`, don't walk
* If current color is `green`, walk
* If current color is `yellow` or `blinking-yellow`, be cautious

In [18]:
from enum import Enum
class Color(Enum):
    Red = 0
    Green = 1
    Yellow = 2
    BlinkingYellow = 3

    @staticmethod
    def all_colors():
        return [Color.Red, Color.Green, Color.Yellow, Color.BlinkingYellow]
    
    @staticmethod
    def text(color):
        return {
            Color.Red: 'red',
            Color.Green: 'green',
            Color.Yellow: 'yellow',
            Color.BlinkingYellow: 'blinking-yellow',
        }.get(color, 'unknown')

In [35]:
class TrafficLight():
    def __init__(self, color):
        self.__curr_color = color
    
    def __next(self):
        return {
            Color.Red: (Color.Green, 500),
            Color.Green: (Color.Yellow, 500),
            Color.Yellow: (Color.BlinkingYellow, 200),
            Color.BlinkingYellow: (Color.Red, 100),
        }.get(self.__curr_color, None)
    
    def update(self):
        next_color, msec = self.__next()
        self.__curr_color = next_color
        return next_color, msec

In [33]:
class Light(Fact):
    """Info about the traffic light."""
    pass


class RobotCrossStreet(KnowledgeEngine):
    @Rule(AS.light << Light(color=Color.Green))
    def green_light(self, light):
        print("Agent: Walk, light is", Color.text(light["color"]))

    @Rule(AS.light << Light(color=Color.Red))
    def red_light(self, light):
        print("Agent: Don't walk, light is", Color.text(light["color"]))

    @Rule(AS.light << Light(color=L(Color.Yellow) | L(Color.BlinkingYellow)))
    def cautious(self, light):
        print("Agent: Be cautious, light is", Color.text(light["color"]))

In [38]:
from time import sleep
engine = RobotCrossStreet()
times = 10
start = choice(Color.all_colors())
print('Starting color:', start)
state_machine = TrafficLight(start)
for i in range(times):
    curr_color, msec = state_machine.update()
    sec = msec / 1000
    sleep(sec)
    print('Current color:', Color.text(curr_color))
    engine.reset()
    engine.declare(Light(color=curr_color))
    engine.run()
    print('-'*100)

Starting color: Color.Green
Current color: yellow
Agent: Be cautious, light is yellow
----------------------------------------------------------------------------------------------------
Current color: blinking-yellow
Agent: Be cautious, light is blinking-yellow
----------------------------------------------------------------------------------------------------
Current color: red
Agent: Don't walk, light is red
----------------------------------------------------------------------------------------------------
Current color: green
Agent: Walk, light is green
----------------------------------------------------------------------------------------------------
Current color: yellow
Agent: Be cautious, light is yellow
----------------------------------------------------------------------------------------------------
Current color: blinking-yellow
Agent: Be cautious, light is blinking-yellow
---------------------------------------------------------------------------------------------------

In [40]:
## Create an ontology factory in order to fetch HPO
from ontobio.ontol_factory import OntologyFactory